### FSM definition

The textbook defines a Finite State Machine (FSM) as a tuple. Think of a _tuple_ as an ordered pair with more than two elements. The tuple for an FSA is $(S,I,s_0,f,g)$ where 
* A finite set of states $S$
* A set of input characters $I$
* A set of output characters $O$
* A start state $s_0$
* A transition function $f:S\times I \rightarrow S$
* An output function $g:S\times I \rightarrow O$


We often draw FSMs by representing each state $s\in S$ with a circle, representing the start state by a circle with an arrow pointing toward it, representing the transition function as labeled arrows connecting one state to another, and representing the output function by adding an additional label to each arrow. Arrow labels are represented as $i, o$ pairs where $i\in I$ is the input character and $o\in O$ is the output character. The label $i$ of the arrow connecting one state to another is the input that causes the FSA to _transition_from one state to another.  The label $o$ on an arrow indicates what is _output_ while the state transition occurs.

---

### Implementations of FSMs ###

The purpose of this tutorial is to explore different ways of implementing state machines in Python.

Consider Figure 2 from Section 13.2.2 of the textbook.
I've modified Figure 2 by changing output set from $O = \{0,1\}$ to $O = \{a,b\}$ because I think it makes it easier to tell what is an input and what is an output.

[![Finite state machine](Table2_Section13.2.2.png)](https://www.dropbox.com/scl/fi/25a8xhn1vojf20pqbeern/Table2_Section13.2.2.png?rlkey=6bhh5j81g1ba03a86os69za4o&dl=0)

Let's look at three different ways to program this FSM.

---

### Method 1: if-then structure ##

The state transition is defined as a function $f: S\times I \rightarrow S$ that maps the present state and input to a next state. If I let $s$ and $i$ denote the state of the machine and the input at the curren time, respectively, and if we let $s'$ denote the next state, then we can write the next state as a function of the present state, $s' = f(s, i)$. 

We can think of this function as an if statement: 

    if the present state is s and the current input is i then the next state is s'

 We can use this idea to create a class for the FSM in the figure above. Let's ignore the output.

In [155]:
############
## Cell 1 ##
############

class StateMachine:
    """ Represent states with strings: 's0', 's1', 's2', 's3' 
    Represent inputs with strings '0', '1' """
    def __init__(self) -> None:
        self._present_state: str = 's0'
        self._states: set[str] = {'s0', 's1', 's2', 's3'}

    def get_next_state(self, input_symbol:str) -> str:
        next_state: str 
        ## Transitions from state s0
        if self._present_state == 's0':
            if input_symbol == '0':
                next_state = 's1'
            elif input_symbol == '1':
                next_state = 's0'
            else: raise ValueError("Illegal input to the state machine " + str(input_symbol))
        ## Transitions from state s1
        elif self._present_state == 's1':
            if input_symbol == '0': 
                next_state = 's3'
            elif input_symbol == '1':
                next_state = 's0'
            else: raise ValueError("Illegal input to the state machine " + str(input_symbol))
        ## Transitions from state s2
        elif self._present_state == 's2':
            if input_symbol == '0': 
                next_state = 's1'
            elif input_symbol == '1':
                next_state = 's2'
            else: raise ValueError("Illegal input to the state machine " + str(input_symbol))
        ## Transitions from state s3
        elif self._present_state == 's3':
            if input_symbol == '0': 
                next_state = 's2'
            elif input_symbol == '1':
                next_state = 's1'
            else: raise ValueError("Illegal input to the state machine " + str(input_symbol))
        ## Illegal state handling
        else: raise ValueError("How did you get into state " + self._present_state)
        return next_state
    
    ### Getters and Setters ###
    def set_state(self, next_state: str) -> None:
        if next_state in self._states: 
            self._present_state = next_state
        else: 
            raise ValueError("Trying to move to illegal state " + next_state)

    def get_state(self) -> str: return self._present_state

The implementation is straightforward. The outer if statements check current state. The inner if statements check the input. And I threw in some error handling just for fun.

Let's see if it works. Consider going through the state transitions by hand for the input 01000. The state sequence should be s0 -> s1 -> s0 -> s1 -> s3 -> s2.

In [156]:
############
## Cell 2 ##
############

fsm: StateMachine = StateMachine()

input_sequence: list[str] = ['0', '1', '0', '0', '0']

print(f"Start state is {fsm.get_state()}")
for symbol in input_sequence:
    present_state:str = fsm.get_state()
    next_state = fsm.get_next_state(symbol)
    fsm.set_state(next_state)
    print(f"Present state: {present_state}, Input: {symbol}, Next state: {next_state}")

Start state is s0
Present state: s0, Input: 0, Next state: s1
Present state: s1, Input: 1, Next state: s0
Present state: s0, Input: 0, Next state: s1
Present state: s1, Input: 0, Next state: s3
Present state: s3, Input: 0, Next state: s2


Although conceptually simple, the code is not modular and would be very difficult to maintain if the state machine were very large. A colleague worked for a company that implemented a FSM as part of a website. The FSM had hundreds of transitions. Nobody dared changed the code for fear of messing up the code.

Let's try an approach that is also conceptually simple and allows us to instantiate any FSM of our choosing.

---

### Method 2: State transition table ###

Let's represent this FSM using something like Table 2. We'll split the table into two pieces: a table that handles state transitions and a table that handles outputs. The first table is a present-state, next-state table. It implements the finite state machine mapping $f:S\times I \rightarrow S$. 

I typed "how do i represent a present state next state table for a finite state machine in python?" into copilot and am using the code from there because it matches what I want to teach. I'm making six changes to the code
 - I'm adding types to each variable, which is what the Python style guide suggests
 - I'm adding comments where I think it will help. 
 - I'm changing the name of the variable _self.state_ to _self.\_present\_state_ since that variable name matches better what we've talked about in class.
 - I'm adding a getter method to access the current state variable. I'm doing this because I like the way that C++ allows us specify whether a class's member variable is public or private. I try to never access a member variable directly and instead use getters and setters.
 - I'm adding a single underscore as a prefix to each class member variable. This is a common convention to indicate that the variable is intended to be private and not used outside of the scope of the class.
 - I'm adding a method that prints out the present-state, next-state table

The basic building block of this implementation is the _transition_ object, which is implemented as a dictionary. More precisely, it is a dictionary of dictionaries. More on that later.

In [157]:
############
## Cell 3 ##
############

class StateMachine:
    def __init__(self) -> None:
        self._present_state: str 
        self._transitions: dict[str,dict[str,str]]= {}

    def add_transition(self, state: str, input_symbol: str, next_state: str) -> None:
        if state not in self._transitions:
            self._transitions[state] = {}
        self._transitions[state][input_symbol] = next_state

    def set_start_state(self, state: str) -> None:
        self._present_state = state

    def get_next_state(self, input_symbol:str) -> str:
        if self._present_state in self._transitions and input_symbol in self._transitions[self._present_state]:
            self._present_state = self._transitions[self._present_state][input_symbol]
        return self._present_state
    
    def get_state(self) -> str: return self._present_state

    def show_table(self) -> None:
        """ Show present state next state table """
        header = f"{'Present State':^15} | {'Input':^8} | {'Next State':^12}"
        line = ""
        for _ in header:
            line += "_"
        print(header)
        print(line)
        for present_state  in self._transitions.keys():
            for input in self._transitions[present_state].keys():
                next_state = self._transitions[present_state][input]
                print(f"{present_state:^15} | {input:^8} | {next_state:^12}")


Let's discuss the line 

    if self._present_state in self._transitions 

Suppose that you have a Python dictionary D = {'a': 1, 'b': 2}. The keys in the dictionary are 'a' and 'b'. When you write 

    if 'a' in D: 

you are asking whether 'a' is in the keys of the dictionary. Let's confirm with an aside.

In [158]:
D = {'a':1, 'b': 2}
if 'a' in D: 
    print('\'a\' is in D')
else:
    print('\'a\' is not in D')

'a' is in D


Try with something that is not a key in the dictionary.

In [159]:
D = {'a':1, 'b': 2}
if 'c' in D: 
    print('\'c\' is in D')
else:
    print('\'c\' is not in D')

'c' is not in D


Returning to the FSM, let's instantiate the class and add the state transitions for the FSM shown above.

In [160]:
############
## Cell 4 ##
############

# Example usage
fsm: StateMachine = StateMachine()

fsm.add_transition('s0', '0', 's1')
fsm.add_transition('s0', '1', 's0')
fsm.add_transition('s1', '0', 's3')
fsm.add_transition('s1', '1', 's0')
fsm.add_transition('s2', '0', 's1')
fsm.add_transition('s2', '1', 's2')
fsm.add_transition('s3', '0', 's2')
fsm.add_transition('s3', '1', 's1')


Let's talk about what each call to _add\_transition_ does. Recall from the definition of a FSM (section 13.2.2, def 1 of the textbook) that a transition is a function that maps present states and inputs to next states, $f: S\times I \rightarrow S$. When the domain and co-domain are finite, we can represent functions as a set of tuples. The tuple $(s0, 0, s1)$ represents what happens to the function $f$ when we pass it $s0$ and $0$. In other words, $f(s0,0) = s1$. Stated simply, the first two elements of each tuple are the present state and input, respectively, and the output is the next state. Thus, each tuple is (present state, input, next state).

Following the textbook, we can represent the transition function using a present state, next state table. What does the present-state, next-state table look like?

In [161]:
############
## Cell 5 ##
############

fsm.show_table()

 Present State  |  Input   |  Next State 
_________________________________________
      s0        |    0     |      s1     
      s0        |    1     |      s0     
      s1        |    0     |      s3     
      s1        |    1     |      s0     
      s2        |    0     |      s1     
      s2        |    1     |      s2     
      s3        |    0     |      s2     
      s3        |    1     |      s1     


Although in a different format than Table 2 in Section 13.2.2, it contains the same information.

We can now specify the start state and do a trace through the FSM.  If the input is 01001 the state sequence should be s0 -> s1 -> s0 -> s1 -> s2. Let's test.

In [162]:
############
## Cell 6 ##
############

fsm.set_start_state('s0')

input_sequence: list[str] = ['0', '1', '0', '0', '0']

print(f"The start state is {fsm.get_state()}")
for symbol in input_sequence:
    present_state:str = fsm.get_state()
    next_state = fsm.get_next_state(symbol)
    print(f"Present state: {present_state}, Input: {symbol}, Next state: {next_state}")


The start state is s0
Present state: s0, Input: 0, Next state: s1
Present state: s1, Input: 1, Next state: s0
Present state: s0, Input: 0, Next state: s1
Present state: s1, Input: 0, Next state: s3
Present state: s3, Input: 0, Next state: s2


The key idea in this implementation is to represent the state transition as a dictionary of dictionaries. The out dictionary is keyed by the present state. Then, the inner dictionary is keyed by the input. 

I like this implementation and wouldn't have a problem if you chose to use something like this on project 1. I prefer a different approach

---

### Method 3: States as Functions ###

The key idea in this third method for representing FSMs is to represent each state as a function and then ask it what should happen when we pass it a given input. For example, we can represent state $s0$ as a function and then ask "state $s0$, what should the next state be if the input is $i$?" Before giving example code, we need to review a programing concept.




Recall from CS 111 that everything is an object in Python: strings, integers, classes, and functions. And every object is assigned to a location in memory. Let me illustrate with a simple function.

In [163]:
def square(x: float) -> float: return x*x

Let's inspect this function.

In [164]:
print(square)

<function square at 0x10c004180>


When you run the cell above you should see something like

    <function square at 0x1158cc2c0>

Let's break down what this means.
- The first thing you see inside the angle brackets is the word _function_. This simply says that the type of the object is a function.
- The second thing you see is the word _square_. This simply says that the name of the function is "square".
- The next thing you see is _at 0x..._. The _0x_ tells you that you're going to see a number written in hexadecimal. (Think of the _x_ in _0x_ as the third letter of hexadecimal.)  The numbers and letters after the _0x_ are the address in memory in which the function is stored. Your numbers will be different than what I show above because the address is assigned when we create the function and depends on what else is in memory on your computer.

Python lets us pass around objects including _function objects_. That means we can pass a function object to another function, return a function from another function, etc. Let's illustrate.

In [165]:
# This import gives us the type hint "function". It is renamed from "Callable"
from typing import Callable as function

def cube(x: float) -> float: return x*x*x
def quad(x: float) -> float: return x*x*x*x

def pick_a_function(choice: str) -> function[[float], float]:
    if choice == "square": return square
    elif choice == "cube": return cube
    elif choice == "quad": return quad
    else: raise ValueError(choice = " is not a valid function type")


Note the import at the top of the block. It just defines the keyword _function_ so that we can tell Python that _pick\_a\_function_ returns a Python function. Let's break down what the following line of code means.

    def pick_a_function(choice: str) -> function[[float], float]:

The line says that we are defining a function called _pick\_a\_function_ that takes a string named _choice_ as input and returns a function. The things in the square brackets next to function

    function[[float], float]

tells us type information about the function being returned. More specifically, the function that is being returned takes as input a float, which is indicated by the _[float]_ and returns an output that is also a float, which is indicated by the _float_ not in brackets. 



Let's call _pick\_a\_function_, observe that it returns a function, and then call the function it calls.

In [166]:
my_function = pick_a_function("square")
print(my_function)
print(f"my function returned {my_function(2)}")


<function square at 0x10c004180>
my function returned 4


The _square_ function was returned. Let's try the other options as well.

In [167]:
my_function = pick_a_function("cube")
print(my_function)
print(f"my function returned {my_function(2)}")

my_function = pick_a_function("quad")
print(my_function)
print(f"my function returned {my_function(2)}")

<function cube at 0x10c0044a0>
my function returned 8
<function quad at 0x10b8efe20>
my function returned 16


So, functions can call and return functions. That can be confusing, but we can use it to define a different kind of FSM. The key idea is that we'll treat each state as a function that takes an input and returns the next state. Let's use that idea to write a FSM that treats states like functions.



Referring to the figure of the FSM above, our job is to create a function _s0_ that we can ask "what will you do on a specific input?" In other words, we let $s_0$ make its own decisions. We pass it an input and it decides where it will go. For now, let's only ask it about the state transition function $f: S\times I \rightarrow S$. We'll add the output function in a later cell.

In [168]:
############
## Cell 7 ##
############

# This import allows us to return a "function". 
# It is renamed from "Callable"
from typing import Callable as function

class StateMachine:
    def __init__(self) -> None:
        self._present_state: function[[str],function] = self.s0
        self._states: set[function[[str], function]] = {self.s0, self.s1, self.s2, self.s3}
    
    ######################################
    ## Define a Function for each state ##
    ######################################
    def s0(self, input_symbol: str) -> function[[str],function]:
        next_state: function
        if input_symbol == "0": next_state = self.s1
        elif input_symbol == "1": next_state = self.s0
        else: raise ValueError("Illegal input to the state machine " + input_symbol)
        return next_state
    
    def s1(self, input_symbol: str) -> function[[str],function]:
        next_state: function
        if input_symbol == "0": next_state = self.s3
        elif input_symbol == "1": next_state = self.s0
        else: raise ValueError("Illegal input to the state machine " + input_symbol)
        return next_state
    
    def s2(self, input_symbol: str) -> function[[str],function]:
        next_state: function
        if input_symbol == "0": next_state = self.s1
        elif input_symbol == "1": next_state = self.s2
        else: raise ValueError("Illegal input to the state machine " + input_symbol)
        return next_state
    
    def s3(self, input_symbol: str) -> function[[str],function]:
        next_state: function
        if input_symbol == "0": next_state = self.s2
        elif input_symbol == "1": next_state = self.s1
        else: raise ValueError("Illegal input to the state machine " + input_symbol)
        return next_state
    
    #########################
    ## Getters and Setters ##
    #########################
    def set_state(self, next_state: function[[str],function]) -> None:
        if next_state in self._states:
            self._present_state = next_state
        else: 
            raise ValueError("Trying to move to illegal state " + next_state)
        
    def get_state(self) -> str: return self._present_state

Notice that this code is almost identical to the if-then-else structure. The only differences are
 - states are represented as functions and not as strings
 - the code is modular

 Let's see if it works. Consider going through the state transitions by hand for the input 01000. The state sequence should be s0 -> s1 -> s0 -> s1 -> s3 -> s2.

In [169]:
fsm: StateMachine = StateMachine()

input_sequence: list[str] = ['0', '1', '0', '0', '0']

present_state: function[[str], tuple[function, str]]
next_state: function[[str], tuple[function, str]]

print(f"The start state is {fsm.get_state()}")
for symbol in input_sequence:
    present_state = fsm.get_state()
    next_state = present_state(symbol) # Ask the present state what the next state should be
    fsm.set_state(next_state)
    print(f"Present state: {present_state}, Input: {symbol}, Next state: {next_state}")

The start state is <bound method StateMachine.s0 of <__main__.StateMachine object at 0x10bf622d0>>
Present state: <bound method StateMachine.s0 of <__main__.StateMachine object at 0x10bf622d0>>, Input: 0, Next state: <bound method StateMachine.s1 of <__main__.StateMachine object at 0x10bf622d0>>
Present state: <bound method StateMachine.s1 of <__main__.StateMachine object at 0x10bf622d0>>, Input: 1, Next state: <bound method StateMachine.s0 of <__main__.StateMachine object at 0x10bf622d0>>
Present state: <bound method StateMachine.s0 of <__main__.StateMachine object at 0x10bf622d0>>, Input: 0, Next state: <bound method StateMachine.s1 of <__main__.StateMachine object at 0x10bf622d0>>
Present state: <bound method StateMachine.s1 of <__main__.StateMachine object at 0x10bf622d0>>, Input: 0, Next state: <bound method StateMachine.s3 of <__main__.StateMachine object at 0x10bf622d0>>
Present state: <bound method StateMachine.s3 of <__main__.StateMachine object at 0x10bf622d0>>, Input: 0, Nex

The state sequence is correct. Of course, the things that are printed out are not strings but rather functions along with information about to whom those functions belong. The word "function" has been replaced by "bound method", which is just Python's way of saying that functions that exist inside classes are called "methods" and they are _bound_ (i.e., belong to) an instantiated class.

---

### Outputs ###

Let's explore outputs. Suppose we have the FSM in the figure above. How would the code need to be modified to track outputs? Let's think about the mental model we are using in the code. We have created a function for each state so that we can ask the function "What should happen when you see a particulare input?" In the cells above, we only asked the function to tell us what the next state should be, i.e., we asked it to implement its part of the transition function $f:S\times I \rightarrow S$. We are now also asking it what the output should be, i.e., we are asking it to implement its part of the output function $G:S\times I \rightarrow O$.

In [170]:
############
## Cell 8 ##
############

class StateMachine:
    def __init__(self) -> None:
        self._present_state: function[[str], function] = self.s0
        self._states: set[function[[str], function]] = {self.s0, self.s1, self.s2, self.s3}
    
    ######################################
    ## Define a Function for each state ##
    ######################################
    def s0(self, input_symbol: str) -> tuple[function[[str], function], str]:
        # return the next state and return the output as a tuple (next state, output)
        next_state: function[[str], function]
        output: str
        if input_symbol == "0": 
            next_state = self.s1
            output = 'b'
        elif input_symbol == "1": 
            next_state = self.s0
            output = 'a'
        else: raise ValueError("Illegal input to the state machine " + input_symbol)
        return (next_state, output)
    
    def s1(self, input_symbol: str) -> tuple[function[[str], function], str]:
        next_state: function[[str], function]
        output: str
        if input_symbol == "0": 
            next_state = self.s3
            output = 'b'
        elif input_symbol == "1": 
            next_state = self.s0
            output = 'b'
        else: raise ValueError("Illegal input to the state machine " + input_symbol)
        return (next_state, output)
    
    def s2(self, input_symbol: str) -> tuple[function[[str], function], str]:
        next_state: function[[str], function]
        output: str
        if input_symbol == "0": 
            next_state = self.s1
            output = 'a'
        elif input_symbol == "1": 
            next_state = self.s2
            output = 'b'
        else: raise ValueError("Illegal input to the state machine " + input_symbol)
        return (next_state, output)
    
    def s3(self, input_symbol: str) -> tuple[function[[str], function], str]:
        next_state: function[[str], function]
        output: str
        if input_symbol == "0": 
            next_state = self.s2
            output = 'a'
        elif input_symbol == "1": 
            next_state = self.s1
            output = 'a'
        else: 
            raise ValueError("Illegal input to the state machine " + input_symbol)
        return (next_state, output)
    
    #########################
    ## Getters and Setters ##
    #########################
    def set_state(self, next_state: function[[str], function]) -> None:
        if next_state in self._states:
            self._present_state = next_state
        else: 
            raise ValueError("Trying to move to illegal state " + next_state)
    def get_state(self) -> str: return self._present_state

In [171]:
fsm: StateMachine = StateMachine()

input_sequence: list[str] = ['0', '1', '0', '0', '0']
output_sequence: list[str] = []
output: str
present_state: function[[str], tuple[function, str]]
next_state: function[[str], tuple[function, str]]

print(f"The start state is {fsm.get_state}")
for symbol in input_sequence:
    present_state= fsm.get_state()
    next_state, output = present_state(symbol) # Ask the present state what the next state and output should be
    fsm.set_state(next_state)
    print(f"Present state: {present_state}, Input: {symbol}, Next state: {next_state}, Output: {output}")
    output_sequence.append(output)

print(output_sequence)


The start state is <bound method StateMachine.get_state of <__main__.StateMachine object at 0x10ba1c790>>
Present state: <bound method StateMachine.s0 of <__main__.StateMachine object at 0x10ba1c790>>, Input: 0, Next state: <bound method StateMachine.s1 of <__main__.StateMachine object at 0x10ba1c790>>, Output: b
Present state: <bound method StateMachine.s1 of <__main__.StateMachine object at 0x10ba1c790>>, Input: 1, Next state: <bound method StateMachine.s0 of <__main__.StateMachine object at 0x10ba1c790>>, Output: b
Present state: <bound method StateMachine.s0 of <__main__.StateMachine object at 0x10ba1c790>>, Input: 0, Next state: <bound method StateMachine.s1 of <__main__.StateMachine object at 0x10ba1c790>>, Output: b
Present state: <bound method StateMachine.s1 of <__main__.StateMachine object at 0x10ba1c790>>, Input: 0, Next state: <bound method StateMachine.s3 of <__main__.StateMachine object at 0x10ba1c790>>, Output: b
Present state: <bound method StateMachine.s3 of <__main__.

---

Let's recap what we've done.
- We implemented a FSM using if-then-else statements. This was based on the mental model that says the transition function operates like "if the present state is _s_ and the input is _i_ then the next state should be _s'_ ".
- We implemented a FSM by constructing a state transition table. This was based on the mental model that says each transition can be encoded as a tuple _(present state, input, next state)_.
- We implemented a FSM by creating a function for each state. This was based on the mental model that says "we can implement the transition function by asking each state (i.e., calling each state function) what its next state and output should be for a given input".

We demonstrated that each of these approaches works fine. I like the last one because (a) it is modular and (b) it allows me to take a state diagram that I've drawn and turn it directly into code. 

When we apply FSMs to project 1, we'll want a generic method that manages our FSM. Specifically, we'll create a method _run_ that
- _manages_ the input 
- _tracks_ the present state
- _runs_ the FSM by asking each state function about the next state and output for a given input
- _sets_ the present state to the next state
- _returns_ the output

Let's also define a State data type so we dont' have to keep writing "function[[str], tuple[function, str]]"

In [172]:
############
## Cell 9 ##
############

VERBOSE = True

# Define State Type #
State = function[[str], tuple[function, str]]

class StateMachine:
    def __init__(self) -> None:
        self.initial_state: State = self.s0

    ######################################
    ## Define a Function for each state ##
    ######################################
    def s0(self, input_symbol: str) -> tuple[function[[str], function], str]:
        # return the next state and return the output as a tuple (next state, output)
        next_state: function[[str], function]
        output: str
        if input_symbol == "0": 
            next_state = self.s1
            output = 'b'
        elif input_symbol == "1": 
            next_state = self.s0
            output = 'a'
        else: raise ValueError("Illegal input to the state machine " + input_symbol)
        return (next_state, output)
    
    def s1(self, input_symbol: str) -> tuple[function[[str], function], str]:
        next_state: function[[str], function]
        output: str
        if input_symbol == "0": 
            next_state = self.s3
            output = 'b'
        elif input_symbol == "1": 
            next_state = self.s0
            output = 'b'
        else: raise ValueError("Illegal input to the state machine " + input_symbol)
        return (next_state, output)
    
    def s2(self, input_symbol: str) -> tuple[function[[str], function], str]:
        next_state: function[[str], function]
        output: str
        if input_symbol == "0": 
            next_state = self.s1
            output = 'a'
        elif input_symbol == "1": 
            next_state = self.s2
            output = 'b'
        else: raise ValueError("Illegal input to the state machine " + input_symbol)
        return (next_state, output)
    
    def s3(self, input_symbol: str) -> tuple[function[[str], function], str]:
        next_state: function[[str], function]
        output: str
        if input_symbol == "0": 
            next_state = self.s2
            output = 'a'
        elif input_symbol == "1": 
            next_state = self.s1
            output = 'a'
        else: raise ValueError("Illegal input to the state machine " + input_symbol)
        return (next_state, output)
    

#############################################
## Define a function that runs the machine ##
#############################################
def run(input_sequence: str, fsm: StateMachine) -> list[str]:
    # Use a C++ style declaration of local variables
    present_state: State = fsm.initial_state
    next_state: State
    output_sequence: list[str] = []

    # Step through each input symbol
    for symbol in input_sequence:
        next_state, output = present_state(symbol)      # Ask the present state what the next state and output should be
        present_state = next_state                # Update the state
        output_sequence.append(output)                  # Record output
        if VERBOSE: 
            print(f"Present state: {present_state}, Input: {symbol}, Next state: {next_state}, Output: {output}")

    # return the output
    return output_sequence


The code above tries to use good modular design. The _StateMachine_ class defines an initial state and all transitions. The _run_ method just does what we've been doing in class: it steps through each state consuming inputs and collecting outputs.

Let's run the fsm on the same input as above.

In [173]:
fsm: StateMachine = StateMachine()
print(run(['0', '1', '0', '0', '0'], fsm))


Present state: <bound method StateMachine.s1 of <__main__.StateMachine object at 0x10bf64dd0>>, Input: 0, Next state: <bound method StateMachine.s1 of <__main__.StateMachine object at 0x10bf64dd0>>, Output: b
Present state: <bound method StateMachine.s0 of <__main__.StateMachine object at 0x10bf64dd0>>, Input: 1, Next state: <bound method StateMachine.s0 of <__main__.StateMachine object at 0x10bf64dd0>>, Output: b
Present state: <bound method StateMachine.s1 of <__main__.StateMachine object at 0x10bf64dd0>>, Input: 0, Next state: <bound method StateMachine.s1 of <__main__.StateMachine object at 0x10bf64dd0>>, Output: b
Present state: <bound method StateMachine.s3 of <__main__.StateMachine object at 0x10bf64dd0>>, Input: 0, Next state: <bound method StateMachine.s3 of <__main__.StateMachine object at 0x10bf64dd0>>, Output: b
Present state: <bound method StateMachine.s2 of <__main__.StateMachine object at 0x10bf64dd0>>, Input: 0, Next state: <bound method StateMachine.s2 of <__main__.Sta

---

### Testing ###

We can use assert statements to test whether the FSM acts like we expect. We'll collect all the assert statements into a single test function. You can ask a LLM what the following code is doing if you've never used assert statements. You can also ask a LLM to explain the difference between a unit test and an integration test.

I created the tests by stepping through the FSM by hand.

In [174]:
VERBOSE = False

def test_fsm():
    ################
    ## Unit tests ##
    ################

    # Transitions from state s0 with input '0'
    next_state, output = fsm.s0('0')
    assert output == 'b', "Output for f(s0, 0) failed"
    assert next_state == fsm.s1, "Next state for f(s0,0) failed"

    # Transitions from state s0 with input '1'
    next_state, output = fsm.s0('1')
    assert output == 'a', "Output for f(s0, 1) failed"
    assert next_state == fsm.s0, "Next state for f(s0,1) failed"

    # Transitions from state s1 with input '0'
    next_state, output = fsm.s1('0')
    assert output == 'b', "Output for f(s1, 0) failed"
    assert next_state == fsm.s3, "Next state for f(s1,0) failed"

    # Transitions from state s1 with input '1'
    next_state, output = fsm.s1('1')
    assert output == 'b', "Output for f(s1, 1) failed"
    assert next_state == fsm.s0, "Next state for f(s1,0) failed"

    # Transitions from state s2 with input '0'
    next_state, output = fsm.s2('0')
    assert output == 'a', "Output for f(s2, 0) failed"
    assert next_state == fsm.s1, "Next state for f(s2,0) failed"

    # Transitions from state s2 with input '1'
    next_state, output = fsm.s2('1')
    assert output == 'b', "Output for f(s2, 1) failed"
    assert next_state == fsm.s2, "Next state for f(s2,0) failed"

    # Transitions from state s3 with input '0'
    next_state, output = fsm.s3('0')
    assert output == 'a', "Output for f(s3, 0) failed"
    assert next_state == fsm.s2, "Next state for f(s3,0) failed"

    # Transitions from state s3 with input '1'
    next_state, output = fsm.s3('1')
    assert output == 'a', "Output for f(s3, 1) failed"
    assert next_state == fsm.s1, "Next state for f(s3,0) failed"

    #######################
    ## Integration tests ##
    #######################
    assert run(['0'], fsm) == ['b'], "Integration test 1 failed"
    assert run(['0', '0'], fsm) == ['b', 'b'], "Integration test 2 failed"
    assert run(['0', '0', '0'], fsm) == ['b', 'b', 'a'], "Integration test 3 failed"
    assert run(['0', '0', '0', '0'], fsm) == ['b', 'b', 'a', 'a'], "Integration test 4 failed"
    assert run(['0', '0', '1', '1'], fsm) == ['b', 'b', 'a', 'b'], "Integration test 5 failed"

    print("All test cases passed")

test_fsm()


All test cases passed


---